# Implementing a model with observed state space components

This notebook shows how to introduce observable characteristics of an individual to the state space. A potential source of unobserved heterogeneity in the models of Keane and Wolpin (1994) and Keane and Wolpin (1997) stems from the fact that individual ability is not observed. The authors try to mitigate the influence by using a finite fixture model with four different types as the years of schooling at the start of the model horizon are potentially not exogenous. If we had data on ability, we could probably shift some of the explanatory power of types to an ability covariate. Furthermore, making type probabilities dependent on the ability level, types become more economically interpretable.

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import respy as rp

## Parameters, options and data

As we have no ability measure in the original data of Keane and Wolpin (1997), we assume that the initial years of schooling serve as a five point scale ability measure. The measure should start at 0 which makes it more suitable to the model.

In [2]:
params, options, df = rp.get_example_model("kw_97_base")

C:\Users\tobia\git\respy\respy\pre_processing\model_processing.py:204: UserWarning: The shares of initial experiences for choice 'school' do not sum to one. Shares are divided by their sum for normalization.
  category=UserWarning,


In [3]:
df["Ability"] = (
    df.groupby("Identifier").Experience_School.transform("first")
    .subtract(7)
    .astype(np.uint8)
)

Furthermore, we include covariates of our new ability measure in the parameter specification and define the covariates in the options. For simplification, we treat our ability measure similar to IQ which was originally used to determine basic mental capabilities. Our single covariate is thus having an ability level higher than zero. Still, we keep the five point scale instead of a simpler two point scale to determine the impact on the size of the state space.

The new covariate enters the wage component of working alternatives and the non-pecuniary component of non-working alternatives positively. 

In [4]:
# Add ability parameters to wage components.
for category in ["wage_white_collar", "wage_blue_collar", "wage_military"]:
    params.loc[(category, "at_least_one_ability"), :] = [
        0.1, "return to having at least ability level one"
    ]

# Add ability parameters to non-pecuniary components.
for category in ["nonpec_school", "nonpec_home"]:
    params.loc[(category, "at_least_one_ability"), :] = [
        2000, "return to having at least ability level one"
    ]

# Add ability parameters to type proobabilities.
for category in ["type_2", "type_3", "type_4"]:
    params.loc[(category, "at_least_one_ability"), :] = [
        0.1, "return to having at least ability level one"
    ]

# Define the probability for ability levels for the simulation.
for name, val in zip(
    [f"ability_{i}" for i in range(5)], [0.0098, 0.0431, 0.201, 0.6702, 0.0759]
):
    params.loc[("observables", name), :] = [
        val, "Probability of having the specified ability level"
    ]

As the ``"name"`` column in the parameter dataframe is matched to covariates, we have to define ``"at_least_one_ability"``. The string under ``options["covariates"]["at_least_one_ability"]`` is evaluated using [pandas.DataFrame.eval()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.eval.html). Under ``options["observables"]["ability"]`` we store the range of ability levels which is five.

In [5]:
options["covariates"]["at_least_one_ability"] = "ability >= 1"

In [6]:
# For simplification we restrict the model to 11 periods.
options["n_periods"] = 11

Here, we solve the model.

In [7]:
state_space = rp.solve(params, options)

In [8]:
params

value  \
category          name                              
delta             delta                    0.7870   
wage_white_collar constant                 8.8043   
                  exp_school               0.0938   
                  exp_white_collar         0.1170   
                  exp_white_collar_square -0.0461   
...                                           ...   
observables       ability_0                0.0098   
                  ability_1                0.0431   
                  ability_2                0.2010   
                  ability_3                0.6702   
                  ability_4                0.0759   

                                                                                     comment  
category          name                                                                        
delta             delta                                                      discount factor  
wage_white_collar constant                 log of rental price if the base skill endowmen...  
                  exp_school               linear return to an additional year of schooli...  
                  exp_white_collar          return to experience, same sector, linear (wage)  
                  exp_white_collar_square  return to experience, same sector, quadratic (...  
...                                                                                      ...  
observables       ability_0                Probability of having the specified ability level  
                  ability_1                Probability of having the specified ability level  
                  ability_2                Probability of having the specified ability level  
                  ability_3                Probability of having the specified ability level  
                  ability_4                Probability of having the specified ability level  

[79 rows x 2 columns]

Here, we calculate the likelihood value of the data.

In [9]:
criterion = rp.get_crit_func(params, options, df)
crit_val = criterion(params)

C:\Users\tobia\git\respy\respy\pre_processing\model_processing.py:204: UserWarning: The shares of initial experiences for choice 'school' do not sum to one. Shares are divided by their sum for normalization.
  category=UserWarning,


In [10]:
crit_val

-55.39840900556615

Here, we simulate a new data set given the proportions of ability levels specified in ``params`` under ``"ability"``.

In [11]:
simulate = rp.get_simulate_func(params, options)
df = simulate(params)

In [13]:
[1, 2, 3] / np.sum(5)

array([0.2, 0.4, 0.6])

In [12]:
df.loc[:, :"Type"].head(20)

,Identifier,Period,Choice,Wage,Experience_Blue_Collar,Experience_Military,Experience_White_Collar,Experience_School,Ability,Type
0,0,0,school,NaN,0,0,0,10,3,2
1,0,1,white_collar,12175.733209,0,0,0,11,3,2
2,0,2,white_collar,28774.143565,0,0,1,11,3,2
3,0,3,blue_collar,18002.634757,0,0,2,11,3,2
4,0,4,school,NaN,1,0,2,11,3,2
5,0,5,white_collar,39528.738378,1,0,2,12,3,2
6,0,6,white_collar,35261.999311,1,0,3,12,3,2
7,0,7,home,NaN,1,0,4,12,3,2
8,0,8,white_collar,24256.351658,1,0,4,12,3,2
9,0,9,home,NaN,1,0,5,12,3,2


## References

> Keane, M. P. and  Wolpin, K. I. (1994). [The Solution and Estimation of Discrete Choice Dynamic Programming Models by Simulation and Interpolation: Monte Carlo Evidence](https://doi.org/10.2307/2109768). *Federal Reserve Bank of Minneapolis*, No. 181.
>
> Keane, M. P. and Wolpin, K. I. (1997). [The Career Decisions of Young Men](https://doi.org/10.1086/262080>). *Journal of Political Economy*, 105(3): 473-522.